### **Data Model**

The goal is to make a DataSet in wich Cycles are realed with GPS and Pressure/Temperature data in sensors. For instance these looks like separeted tables but since we hace the exact TimeStamp in wich the medition are made we can create artificial keys in order to join the tables.

##### **Two submodels can be made**
- ```Cicles -> GPS```
- ```Tires -> Sensors```

For TKPH calculation we need load, distance and time wich are on "Cicles" table. But also temperature that is in Sensors table, so we need to make a complete model in wich data from each cicle are going to be inherited to the tires and to the sensors records. Information from GPS is not needed to add also will be difficult since sensors and gps records are not coordinated.

- ```Clicles -> Tires -> Sensors```


In [106]:
#  Libraries
import pandas as pd
import sqldf
import uuid
from datetime import datetime

In [107]:
# Reading Data
gps_vehicle_dtu003 = pd.read_excel('data/gps_vehicle_DTU003.xlsx')
gps_vehicle_dtu004 = pd.read_excel('data/gps_vehicle_DTU004.xlsx')

with open('querys/staging/stg_gps_vehicles.sql', 'r') as fp:
    gps_vehicle_query = fp.read()

vehicle_gps = sqldf.run(gps_vehicle_query)
vehicle_gps.head(2)

,Time,Vehicle,Vehicle Type,Latitude,Longitude,Altitude (m),Speed (km/h)
0,2023-04-03 00:00:01,DTU003,LIEBHERR-T284,8.844217,-80.664267,185.1,21
1,2023-04-03 00:00:06,DTU003,LIEBHERR-T284,8.844047,-80.664377,184.3,16


In [108]:
# Reading Data
treps_vehicle_dtu003 = pd.read_excel('data/treps_vehicle_DTU003.xlsx')
treps_vehicle_dtu004 = pd.read_excel('data/treps_vehicle_DTU004.xlsx')

with open('querys/staging/stg_treps_vehicles.sql', 'r') as fp:
    treps_vehicle_query = fp.read()

vehicle_treps = sqldf.run(treps_vehicle_query)
vehicle_treps.head(2)

,Time,Vehicle,Vehicle Type,Position,Tire Brand,Tire SN,Tire Type,Tag SN,Pressure (psi),Temp (C),Recommended Pressure (psi),Alarm Status,Red Temp (C),Amber Temp (C),Red Low Pressure (psi),Amber Low Pressure (psi),Red High Pressure (psi),Amber High Pressure (psi)
0,2023-04-03 00:00:02,DTU003,LIEBHERR-T284,5,BRIDGESTONE,B1BSG0062,59/80R63VRPSE2AE4,31752294,117,58,116,No Alarm,85,80,102,108,130,124
1,2023-04-03 00:00:31,DTU003,LIEBHERR-T284,2,BRIDGESTONE,B2M7G0132,59/80R63VRPSE2AE4,31762560,117,57,116,No Alarm,85,80,102,107,129,124


In [109]:
vehicle_cicles = pd.read_excel('data/Kaltire Total Kilometers Roaded - Cobre Panama.xlsx')

with open('querys/staging/stg_vehicle_cicles.sql', 'r') as fp:
    stg_vehicle_cicles_query = fp.read()

vehicle_cicles = sqldf.run(stg_vehicle_cicles_query)
vehicle_cicles.head(2)

Kaltire Total Kilometers Roaded


,Vehicle,Cycle Date,Organization,Start Time,End Time,Empty Travel Time,Empty Stopped Time,Load Time,Stopped Loaded Time,Operating Hours,Kilometers Loaded,Kilometers Empty,Payload,Operator,Loading Point,Dumping Point,Haul Distance,Fuel Used,Loader Passes
0,DTU037,2023-03-04 07:01:41,Minera Panama,2023-03-04 07:00:00,2023-03-04 07:01:41,0.015000,0,0.0,0,0.000000,0.0,0.057821,321.0,Paulo Gonzalez,CO_N24_P180,WSD_RDM_N28_P165,0,12.983962,SHE05
1,DTU036,2023-03-04 07:03:19,Minera Panama,2023-03-04 07:00:00,2023-03-04 07:03:19,0.018611,0,0.0,0,0.055278,0.0,0.072407,336.0,Joel Enrique Reyes Arauz,BO_T37_P105,WSD_MSA_X30_P090,0,47.961168,SHE02


## **Modeled Tables**

In [110]:
# Vehicles Table
vehicle_cicles["uuid"] = vehicle_cicles.apply(lambda _: str(uuid.uuid4()), axis = 1)
vehicle_cicles.set_index(["uuid"])

kaltire_date_format = "%Y-%m-%d %H:%M:%S"

vehicle_gps["Time"] = vehicle_gps["Time"].apply(lambda x: datetime.strptime(str(x), "%Y-%d-%m %H:%M:%S"))

vehicle_cicles["Start Time"] = vehicle_cicles["Start Time"].apply(lambda x: datetime.strptime(str(x), kaltire_date_format))
vehicle_cicles["End Time"] = vehicle_cicles["End Time"].apply(lambda x: datetime.strptime(str(x), kaltire_date_format))
vehicle_cicles["Cycle Date"] = vehicle_cicles["Cycle Date"].apply(lambda x: datetime.strptime(str(x), kaltire_date_format))



with open('querys/modeled/vehicle_gps.sql', 'r') as fp:
    vehicle_gps_query = fp.read()

vehicle_gps = sqldf.run(vehicle_gps_query)
vehicle_gps.head(2)

,cicle_uuid,Time,Latitude,Longitude,Altitude (m),Speed (km/h)
0,a077320c-513d-455b-bba5-44dd83829f73,2023-03-04 07:00:03,8.85451,-80.671938,117.8,0
1,a077320c-513d-455b-bba5-44dd83829f73,2023-03-04 07:00:08,8.85451,-80.671938,117.8,0


In [111]:
# OPTPUTS

vehicle_gps.to_excel('outputs/vehicle_gps.xlsx')
vehicle_treps.to_excel('outputs/vehicle_treps.xlsx')
vehicle_cicles.to_excel('outputs/vehicle_cicles.xlsx')